# Objective

* Predict whether the customer ( Applicant ) loan will be Reject ( Bad ) or Approved ( Good )

* Yes = loan Reject , No =loan approved

In [34]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O 
import os
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

################# visualization #################
import matplotlib.pyplot as plt 
from PIL import  Image
import seaborn as sns 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

# 1. Data Importing

In [35]:
import pandas as pd
Bank = pd.read_csv (r"C:\Users\shubh\Desktop\Machine Learning\Bank_Loan.csv")

#first few rows
Bank.head()

,Loan_ID,Age,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,LoanAmount,Previous_Loan_Taken,Cibil_Score,Property_Area,Customer_Bandwith,Tenure,Loan_Status
0,LP001002,43,Male,No,3,Graduate,No,54005,334113,Yes,668,Urban,Good,43,No
1,LP001003,57,Male,Yes,2,Graduate,No,28817,736026,Yes,684,Rural,Good,13,Yes
2,LP001005,25,Male,Yes,3,Graduate,Yes,99939,712432,Yes,788,Urban,Good,10,No
3,LP001006,44,Male,Yes,1,Not Graduate,No,38470,267161,Yes,888,Urban,Good,9,No
4,LP001008,58,Male,No,1,Graduate,No,28991,402474,Yes,703,Urban,Good,11,No


# 2. Data Pre-Processing 

1. Data Type and conversion
2. Identifying & Treatment Missing Value 

#### 2.1. Data Type and conversion

In [36]:
Bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Loan_ID              981 non-null    object
 1   Age                  981 non-null    int64 
 2   Gender               957 non-null    object
 3   Married              978 non-null    object
 4   Dependents           981 non-null    int64 
 5   Education            981 non-null    object
 6   Self_Employed        926 non-null    object
 7   ApplicantIncome      981 non-null    int64 
 8   LoanAmount           981 non-null    int64 
 9   Previous_Loan_Taken  981 non-null    object
 10  Cibil_Score          981 non-null    int64 
 11  Property_Area        981 non-null    object
 12  Customer_Bandwith    981 non-null    object
 13  Tenure               981 non-null    int64 
 14  Loan_Status          981 non-null    object
dtypes: int64(6), object(9)
memory usage: 115.1+ KB


#### 2.2. Identifying & Treatment Missing Value

In [37]:
# count the number of NaN values in each column
Bank.isnull().sum()

Loan_ID                 0
Age                     0
Gender                 24
Married                 3
Dependents              0
Education               0
Self_Employed          55
ApplicantIncome         0
LoanAmount              0
Previous_Loan_Taken     0
Cibil_Score             0
Property_Area           0
Customer_Bandwith       0
Tenure                  0
Loan_Status             0
dtype: int64

#### Replacing Missing value by Mode

In [38]:
Bank.SeniorCitizen.value_counts(dropna=False)

AttributeError: 'DataFrame' object has no attribute 'SeniorCitizen'

In [ ]:
Bank["Married"].mode()[0]

In [ ]:
for column in ['Married','Gender','Self_Employed']:
    Bank[column].fillna(Bank[column].mode()[0], inplace=True)

In [ ]:
Bank.Married.value_counts(dropna=False)

In [ ]:
# count the number of NaN values in each column
print(Bank.isnull().sum())

In [ ]:
Bank.head()

In [ ]:
#Tenure to categorical column
def tenure_lab(churn) :
    if Bank["Tenure"] <= 6 :
        return "Tenure_0_6"
    elif (Bank["Tenure"] > 6) & (Bank["Tenure"] <= 12 ):
        return "Tenure_7_12"
    elif (Bank["Tenure"] > 12) & (Bank["Tenure"] <= 24) :
        return "Tenure_13_24"
    elif (Bank["Tenure"] > 24) & (Bank["Tenure"] <= 36) :
        return "Tenure_25-36"
    elif Bank["Tenure"] > 36 :
        return "Tenure_gt_36"
# banK = x
Bank["Tenure_group"] = Bank.apply(lambda x :tenure_lab(x),axis = 1)

In [ ]:
Bank.head()

In [ ]:
sns.boxplot(Bank.Age)

In [ ]:
Bank.Tenure_group.value_counts(dropna=False)

In [ ]:
((267+266+259)/981)*100

* 80% of customer required loan tenure for more than a Year

# Data Visualization

#### Bad Rate Analysis 

In [ ]:
Bank.Loan_Status.value_counts()

In [ ]:
(269/981)*100 # 27% is bad rate in data 

In [ ]:
b

### b

In [ ]:
Bank_Yes= Bank[Bank["Loan_Status"] == "Yes"]  # 269

Bank_Yes

In [ ]:
import plotly.express as px
fig = px.sunburst(Bank_Yes, path=["Gender",'Education','Self_Employed',
                                    'Tenure_group','Property_Area'])
fig.show()

### Interpetation : - Trend Analysis
* Customer ==  Male,Graduate,salaried & Tenure  less than a year , Rural   == Loan Rejected 
* customer ==  Male,Graduate,salaried & Tenure  less than a year ,Urban   == Loan Rejected 
* customer ==  Female,Graduate,salaried & Tenure <= 6 Months ,Rural    == Loan Rejected 



# 5. Breaking Data into Two Part
 * Quantitative Variable
 * Qualitative Variable
 


In [ ]:
Bank=Bank.drop('Loan_ID',axis=1)
Bank.head()

In [ ]:
Bank.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
# Qualitative Variable
Quantitative_Variable = Bank[Bank.select_dtypes(include=[np.number]).columns.tolist()]
Quantitative_Variable.head(3)

In [ ]:
Qualitative_Variable = Bank[Bank.select_dtypes(include=['object']).columns.tolist()]
Qualitative_Variable.head(3)

### Converting qualitative Variable into number

In [ ]:
from sklearn.preprocessing import LabelEncoder
Qualitative_Variable=Qualitative_Variable.apply(LabelEncoder().fit_transform) # label in ascending order
Qualitative_Variable.head()

## Combining the data set 

In [ ]:
Final_combined = pd.concat([Qualitative_Variable, Quantitative_Variable],axis=1)
Final_combined.head()

# Data Partition 

In [ ]:
from sklearn.model_selection import train_test_split

# define our input variable (X) & output variable
X = Final_combined.drop('Loan_Status', axis = 1)
Y = Final_combined[['Loan_Status']]
# Split X and y into X_
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=56)

# Model building 

# Hypothesis

* H0 :- There is no log linear relationship between Loan_Status and all independent Variable

Vs 

* H1 :- There is log linear relationship between Loan_Status and all independent Variable

Alpha = 0.05 (5%)

# Method 1 - Logistics Regression

#Needed to run the logistic regression
import statsmodels.formula.api as smf
result=smf.logit("Loan_Status~C(Gender)+C(Married)+C(Education)+C(Self_Employed)+C(Previous_Loan_Taken)+C(Property_Area)+C(Customer_Bandwith)+C(Tenure_group)+Age+Dependents+LoanAmount+Cibil_Score+ApplicantIncome",data=train).fit()
print(result.summary2())

# Method 2 - Logistics Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
logreg = LogisticRegression()
output=logreg.fit(x_train, y_train) # model

In [ ]:
output.intercept_  # beto 

In [ ]:
output.coef_  # Beta 1 -- 

In [ ]:
output.feature_names_in_

* We Reject Ho
* There is log linear relationship between Loan_Status and Married,Previous_Loan_Taken,Customer_Bandwith,Cibil_Score

# Step 6 : Predictions on Train Dataset

In [ ]:
train=pd.concat([X_train,y_train],axis=1)
train.head()

In [ ]:
train['Predicted']=output.predict(X_train)
train.head()

train['Predicted']=np.where(train['Probability'] >= 0.7,1,0) 
train.head()

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(train['Predicted'], train['Loan_Status'])
matrix

In [ ]:
Accuracy_Train=((555+43)/(784)*100)
print(Accuracy_Train)

* Accuracy of Model is 78% , So we say Model good fit data  

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(train['Loan_Status'],train['Predicted']))

* Accuracy of  Model is 76%,Hence Model is good Fit 
* Accuracy of Bad Customer Capture by Model is 19% ( Sensitivity )
* Accuracy of Good Customer Capture by Model is 99% (Specificity)

# Step 8 : Predictions on Test Dataset

In [ ]:
test=pd.concat([X_test,y_test],axis=1)
test.head()

In [ ]:
test['Predicted']=output.predict(X_test)
test.head()

test['']=np.where(test['Probability'] >= 0.7,1,0) 
test.head()

# Step 9 : Model Performance Metrics on Test data 

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(test['Predicted'],test['Loan_Status'])
print(matrix)

In [ ]:
Accuracy_test=((149+8)/(197)*100)
Accuracy_test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test['Loan_Status'],test['Predicted']))

* Accuracy of  Model is 81%,Hence Model is good Fit 
* Accuracy of Bad Customer Capture by Model is 21% ( Sensitivity )
* Accuracy of Good Customer Capture by Model is 100% 

# Coefficient 

* For future Prediction below 4 variable data is mandatory 

* Note :- If anyone is not present output will not be reliable 

In [ ]:
Model_Values=pd.DataFrame(output.intercept_,index=['Intercept'],columns=['Coefficient'])
Model_Values

In [ ]:
rows=['Gender', 'Married', 'Education', 'Self_Employed',
       'Previous_Loan_Taken', 'Property_Area', 'Customer_Bandwith',
       'Tenure_group', 'Age', 'Dependents', 'ApplicantIncome', 'LoanAmount',
       'Cibil_Score', 'Tenure']

abc=pd.DataFrame(output.coef_.T,index=rows,columns=['Coefficient'])
abc

In [ ]:
Model_Values.append(abc)

# Finish